In [3]:
import pandas as pd

In [2]:
import os

In [4]:
report_path = "E:\\SCHOOL_EMAIL_SENDER\\CREWAI_AGENT\\report.md"
if os.path.exists(report_path):
    with open(report_path, "r", encoding="utf-8") as f:
        report_content = f.read()

In [5]:
report_content

'# Student Performance Report\n## Summary\nThe student performance report is based on the analysis of the outputs from Task1, Task2, and Task3. The report provides a comprehensive overview of student performance, highlighting areas of strength and weakness, and informing strategies to support student success.\n\n## Subject Performance\nThe subject-wise performance of students is as follows:\n- Tamil: 4 students passed with 80%+ attendance\n- English: 3 students passed with 80%+ attendance\n- Maths: 2 students passed with 80%+ attendance\n- Science: 1 student passed with 80%+ attendance\n- Social: 1 student passed with 80%+ attendance\n\n## Attendance Analysis\nThe attendance records show that 4 students have 80%+ attendance, while 6 students have less than 80% attendance.\n\n## Actions Taken Summary\nThe following actions have been taken:\n- Identified areas where students need support\n- Provided targeted interventions to promote student success\n- Reviewed attendance records to ident

In [4]:
df = pd.read_csv("E:\\SCHOOL_EMAIL_SENDER\\CREWAI_AGENT\\Inputs\\data.csv")
print(df)

   Roll_No student_name  Tamil  English  Maths  Science  Social  \
0        1        Aarav     54       84     54       60     100   
1        2         Diya     54       49     85       51      83   
2        3      Karthik     71       77     80       37      82   
3        4        Meena     44       86     68      100      72   
4        5         Ravi     38       63     43       93      45   
5        6      Lakshmi     47       95     53       38      71   
6        7       Vikram     58       96     69       65      50   
7        8        Priya     55       91     98       72      66   
8        9        Rahul     67       65     71       92      71   
9       10        Sneha     66       50     62       95      51   

             parent_email  attendance_Percentage  
0  nkumar861019@gmail.com                  69.19  
1  mkumar861019@gmail.com                  75.12  
2  nkumar861019@gmail.com                  96.19  
3  mkumar861019@gmail.com                  62.35  
4  nkum

In [3]:
df.columns

Index(['Roll_No', 'student_name', 'Tamil', 'English', 'Maths', 'Science',
       'Social', 'parent_email', 'attendance_Percentage', 'pass_all', 'Total',
       'percentage_ok'],
      dtype='object')

In [5]:
store = {}
Roll_no = student_name = email_id = Total = Atendance_percentage = None
for cols in df.columns.tolist():
    col = cols.strip().lower()
    if "roll" in col or "no" in col or "number" in col:
        Roll_no = cols
        store[cols] = cols
    elif "student" in col or "name" in col:
        student_name = cols
        store[cols] = cols
    elif "email" in col or "parent" in col:
        email_id = cols
        store[cols] = cols
    elif "total" in col or "marks" in col:
        Total = cols
        store[cols] = cols
    elif "attendance" in col or "percentage" in col:
        Atendance_percentage = cols
        store[cols] = cols

print("Identified Columns:", store)
print("values:", store.values())
numeric_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()
print("Numeric Columns:", numeric_cols)
cols = [col for col in numeric_cols if col not in store.values()]
subjects = cols

print("Subject Column Found:", subjects)        

for i in range(len(df[subjects])):
    print(df[student_name][i])
    print(df[subjects].iloc[i])

Identified Columns: {'Roll_No': 'Roll_No', 'student_name': 'student_name', 'parent_email': 'parent_email', 'attendance_Percentage': 'attendance_Percentage', 'Total': 'Total', 'percentage_ok': 'percentage_ok'}
values: dict_values(['Roll_No', 'student_name', 'parent_email', 'attendance_Percentage', 'Total', 'percentage_ok'])
Numeric Columns: ['Roll_No', 'Tamil', 'English', 'Maths', 'Science', 'Social', 'attendance_Percentage', 'Total']
Subject Column Found: ['Tamil', 'English', 'Maths', 'Science', 'Social']
Karthik
Tamil      71
English    77
Maths      80
Science    37
Social     82
Name: 0, dtype: int64
Ravi
Tamil      38
English    63
Maths      43
Science    93
Social     45
Name: 1, dtype: int64
Lakshmi
Tamil      47
English    95
Maths      53
Science    38
Social     71
Name: 2, dtype: int64
Rahul
Tamil      67
English    65
Maths      71
Science    92
Social     71
Name: 3, dtype: int64


In [16]:
val = df[subjects].iloc[2].to_dict()

In [17]:
print("\n".join([f"{sub}: {val}" for sub,val in val.items()]))

Tamil: 47
English: 95
Maths: 53
Science: 38
Social: 71


In [9]:
# --- Send emails ---
for i in range(len(df)):
    name = df[student_name][i]
    email = df[email_id][i]
    attendance = df[Atendance_percentage][i]
    total = df[Total][i]
    subject_marks = df[subjects].iloc[i].to_dict()
    average = total / len(subject_marks)

    # Format marks as a string
    marks_str = "\n".join([f"{sub}: {mark}" for sub, mark in subject_marks.items()])

    # Email content
    message = f"""
Dear Parent,

Your child {name} has passed all subjects and maintained {attendance}% attendance.

Marks Details:
{marks_str}

Total: {total}
Average: {average:.2f}%

Keep up the great performance!

Best regards,
School Administration
"""

In [ ]:
from crewai_tools import tool
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

@tool("custom_tool2")
def send_pass_mails(file_path: str):
    """
    Tool to send congratulatory emails to parents for students who passed all subjects
    and maintained attendance >= 80%.
    """
    df = pd.read_csv(file_path)

    # --- Identify key columns dynamically ---
    store = {}
    Roll_no = student_name = email_id = Total = Attendance_percentage = None
    for cols in df.columns.tolist():
        col = cols.strip().lower()
        if "roll" in col or "no" in col or "number" in col:
            Roll_no = cols; store[cols] = cols
        elif "student" in col or "name" in col:
            student_name = cols; store[cols] = cols
        elif "email" in col or "parent" in col:
            email_id = cols; store[cols] = cols
        elif "total" in col or "marks" in col:
            Total = cols; store[cols] = cols
        elif "attendance" in col or "percentage" in col:
            Attendance_percentage = cols; store[cols] = cols

    # Identify subjects (numeric columns not already in store)
    numeric_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()
    subjects = [col for col in numeric_cols if col not in store.values()]

    print("Identified Columns:", store)
    print("Subjects:", subjects)

    # --- Email setup ---
    sender_email = "mnithish1231234@gmail.com"
    sender_password = "aors panv cmjc kgti"

    # Create SMTP session
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(sender_email, sender_password)

    # --- Send emails ---
    for i in range(len(df)):
        name = df[student_name][i]
        email = df[email_id][i]
        attendance = df[Attendance_percentage][i]
        total = df[Total][i]
        subject_marks = df[subjects].iloc[i].to_dict()
        average = total / len(subject_marks)

        # Format marks as a string
        marks_str = "\n".join([f"{sub}: {mark}" for sub, mark in subject_marks.items()])

        # Email content
        message = f"""
Dear Parent,

Your child {name} has passed all subjects and maintained {attendance}% attendance.

Marks Details:
{marks_str}

Total: {total}
Average: {average:.2f}%

Keep up the great performance!

Best regards,
School Administration
"""

        # Prepare email
        msg = MIMEMultipart()
        msg["From"] = sender_email
        msg["To"] = email
        msg["Subject"] = "Student Performance Report"
        msg.attach(MIMEText(message, "plain"))

        # Send
        server.sendmail(sender_email, email, msg.as_string())
        print(f"✅ Email sent successfully to {name} ({email})")

    # Close SMTP connection
    server.quit()

    return {"message": "Emails sent successfully to all passing students."}


In [3]:
df = pd.read_csv("E:\\SCHOOL_EMAIL_SENDER\\CREWAI_AGENT\\Outputs\\fail_in_pass_or_attendance.csv")

In [4]:
store = {}
Roll_no = student_name = email_id = Total = Attendance_percentage = None

for cols in df.columns.tolist():
    col = cols.strip().lower()
    if "roll" in col or "no" in col or "number" in col:
        Roll_no = cols
        store[cols] = cols
    elif "student" in col or "name" in col:
        student_name = cols
        store[cols] = cols
    elif "email" in col or "parent" in col:
        email_id = cols
        store[cols] = cols
    elif "total" in col or "marks" in col:
        Total = cols
        store[cols] = cols
    elif "attendance" in col or "percentage" in col:
        Attendance_percentage = cols
        store[cols] = cols

numeric_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()
cols = [col for col in numeric_cols if col not in store.values()]
subjects = cols

In [5]:
subjects

['Tamil', 'English', 'Maths', 'Science', 'Social']

In [15]:
for i in range(len(df)):
    name = df[student_name][i]
    email = df[email_id][i]
    attendance = df[Attendance_percentage][i] if Attendance_percentage else 0
    total = df[Total][i] if Total else 0
    subject_marks = df[subjects].iloc[i].to_dict()

In [16]:
# Analyze failure reasons
failed_subjects = {sub: mark for sub, mark in subject_marks.items() if mark < 35}
attendance_low = attendance < 80

In [23]:
detail = {"name":"NithishKumar","age": 23,"District": "Ariyalur","Domain":"AI_ML"}

In [28]:
for i in detail.keys():
    print(i)

name
age
District
Domain


In [29]:
email_count = 0
attendance_issues = 0
marks_issues = 0
both_issues = 0

detailed_report = []

In [30]:
has_marks_issue = len(failed_subjects) > 0
has_attendance_issue = attendance_low

if has_marks_issue and has_attendance_issue:
    both_issues += 1
    issue_type = "BOTH ATTENDANCE AND ACADEMIC PERFORMANCE"
elif has_marks_issue:
    marks_issues += 1
    issue_type = "ACADEMIC PERFORMANCE"
else:
    attendance_issues += 1
    issue_type = "ATTENDANCE"

In [39]:
has_attendance_issue

np.True_

In [25]:
len(detail.values())

4

In [31]:
message_parts = [
    f"Dear Parent/Guardian,",
    f"",
    f"This is an important notice regarding your child, {name}.",
    f"",
    f"⚠️ WARNING: REQUIRES IMMEDIATE ATTENTION ⚠️",
    f"",
    f"Issue Category: {issue_type}",
    f"",
]


In [32]:
if has_attendance_issue:
    message_parts.extend([
        f"📊 ATTENDANCE CONCERN:",
        f"Current Attendance: {attendance:.2f}% (Required: 80% minimum)",
        f"Status: BELOW REQUIRED THRESHOLD",
        f"",
    ])


In [33]:
message_parts

['Dear Parent/Guardian,',
 '',
 'This is an important notice regarding your child, Sneha.',
 '',
 '⚠️ WARNING: REQUIRES IMMEDIATE ATTENTION ⚠️',
 '',
 'Issue Category: ATTENDANCE',
 '',
 '📊 ATTENDANCE CONCERN:',
 'Current Attendance: 0.00% (Required: 80% minimum)',
 'Status: BELOW REQUIRED THRESHOLD',
 '']

In [34]:
if has_marks_issue:
    message_parts.extend([
        f"📚 ACADEMIC PERFORMANCE CONCERN:",
        f"The following subject(s) require immediate attention:",
        f"",
    ])

In [35]:
message_parts

['Dear Parent/Guardian,',
 '',
 'This is an important notice regarding your child, Sneha.',
 '',
 '⚠️ WARNING: REQUIRES IMMEDIATE ATTENTION ⚠️',
 '',
 'Issue Category: ATTENDANCE',
 '',
 '📊 ATTENDANCE CONCERN:',
 'Current Attendance: 0.00% (Required: 80% minimum)',
 'Status: BELOW REQUIRED THRESHOLD',
 '']

In [8]:
message = "\n".join(message_parts)

In [7]:
print(message,end="\n\n")

Dear Parent/Guardian,

This is an important notice regarding your child, Sneha.

⚠️ WARNING: REQUIRES IMMEDIATE ATTENTION ⚠️

Issue Category: ATTENDANCE

📊 ATTENDANCE CONCERN:
Current Attendance: 0.00% (Required: 80% minimum)
Status: BELOW REQUIRED THRESHOLD

Complete Performance Report:
  • Tamil: 66/100 - ✓ PASS
  • English: 50/100 - ✓ PASS
  • Maths: 62/100 - ✓ PASS
  • Science: 95/100 - ✓ PASS
  • Social: 51/100 - ✓ PASS

Total Marks: 324

Average Marks: 64.80%


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

⚠️ IMPORTANT WARNING ⚠️

If these issues are not addressed and improved in the upcoming evaluation period:

1. The student may face academic probation
2. Promotion to the next grade may be at risk
3. The school will NOT take responsibility for any future academic consequences
4. Additional remedial classes may be mandated at extra cost
5. The student may be required to repeat the current academic year

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

IMMEDIATE ACTIONS REQUIRED:

•

In [6]:
from crewai.tools import tool
import pandas as pd
import os


path_name = "E:\\SCHOOL_EMAIL_SENDER\\CREWAI_AGENT\\Outputs\\fail_in_pass_or_attendance.csv"
df = pd.read_csv(path_name)

store = {}
Roll_no = student_name = email_id = Total = Attendance_percentage = None

for cols in df.columns.tolist():
    col = cols.strip().lower()
    if "roll" in col or "no" in col or "number" in col:
        Roll_no = cols
        store[cols] = cols
    elif "student" in col or "name" in col:
        student_name = cols
        store[cols] = cols
    elif "email" in col or "parent" in col:
        email_id = cols
        store[cols] = cols
    elif "total" in col or "marks" in col:
        Total = cols
        store[cols] = cols
    elif "attendance" in col:
        Attendance_percentage = cols
        store[cols] = cols

numeric_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()
cols = [col for col in numeric_cols if col not in store.values()]
subjects = cols


# Counters for report
email_count = 0
attendance_issues = 0
marks_issues = 0
both_issues = 0

detailed_report = []

# --- Send warning emails ---
for i in range(len(df)):
    name = df[student_name][i]
    email = df[email_id][i]
    attendance = df[Attendance_percentage][i] if Attendance_percentage else 0
    total = df[Total][i] if Total else 0
    subject_marks = df[subjects].iloc[i].to_dict()
    
    # Analyze failure reasons
    failed_subjects = {sub: mark for sub, mark in subject_marks.items() if mark < 35}
    attendance_low = attendance < 80
    print("attendance_low:",attendance_low)
    # Determine issue type
    has_marks_issue = len(failed_subjects) > 0
    has_attendance_issue = attendance_low

    # if i == 0:
    #     print(f"📊 DEBUG - First student data:")
    #     print(f"  Name: {name}")
    #     print(f"  Attendance: {attendance}")
    #     print(f"  Total: {total}")
    #     print(f"  Marks: {subject_marks}\n")
    
    if has_marks_issue and has_attendance_issue:
        both_issues += 1
        issue_type = "BOTH ATTENDANCE AND ACADEMIC PERFORMANCE"
    elif has_marks_issue:
        marks_issues += 1
        issue_type = "ACADEMIC PERFORMANCE"
    else:
        attendance_issues += 1
        issue_type = "ATTENDANCE"

    # Build email body based on issues
    message_parts = [
        f"Dear Parent/Guardian,",
        f"",
        f"This is an important notice regarding your child, {name}.",
        f"",
        f"⚠️ WARNING: REQUIRES IMMEDIATE ATTENTION ⚠️",
        f"",
        f"Issue Category: {issue_type}",
        f"",
    ]

    # Attendance issue details
    if has_attendance_issue:
        message_parts.extend([
            f"📊 ATTENDANCE CONCERN:",
            f"Current Attendance: {attendance:.2f}% (Required: 80% minimum)",
            f"Status: BELOW REQUIRED THRESHOLD",
            f"",
        ])

    # Academic performance details
    if has_marks_issue:
        message_parts.extend([
            f"📚 ACADEMIC PERFORMANCE CONCERN:",
            f"The following subject(s) require immediate attention:",
            f"",
        ])
        
        for subject, mark in failed_subjects.items():
            message_parts.append(f"  • {subject}: {mark}/100 (Required: 35 minimum) - FAILED")
        
        message_parts.extend([
            f"",
            f"Total Marks: {total}",
            f"",
        ])

    # All subject marks for reference
    message_parts.extend([
        f"Complete Performance Report:",
    ])
    for subject, mark in subject_marks.items():
        status = "✓ PASS" if mark >= 35 else "✗ FAIL"
        message_parts.append(f"  • {subject}: {mark}/100 - {status}")
    
    message_parts.extend([
            f"",
            f"Total Marks: {total}",
            f"",
            f"Average Marks: {total / len(subject_marks):.2f}%",
            f"",
        ])
    
    # Warning and consequences
    message_parts.extend([
        f"",
        f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━",
        f"",
        f"⚠️ IMPORTANT WARNING ⚠️",
        f"",
        f"If these issues are not addressed and improved in the upcoming evaluation period:",
        f"",
        f"1. The student may face academic probation",
        f"2. Promotion to the next grade may be at risk",
        f"3. The school will NOT take responsibility for any future academic consequences",
        f"4. Additional remedial classes may be mandated at extra cost",
        f"5. The student may be required to repeat the current academic year",
        f"",
        f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━",
        f"",
        f"IMMEDIATE ACTIONS REQUIRED:",
        f"",
    ])

    if has_attendance_issue:
        message_parts.extend([
            f"• Ensure regular and punctual attendance",
            f"• Provide valid medical certificates for any absences",
            f"• Contact the school administration to discuss attendance improvement plan",
        ])

    if has_marks_issue:
        message_parts.extend([
            f"• Schedule a meeting with subject teachers",
            f"• Enroll in remedial/tutoring programs",
            f"• Establish a structured study routine at home",
            f"• Monitor homework and assignment completion daily",
        ])

    message_parts.extend([
        f"",
        f"Please contact the school office within 7 days to discuss an improvement plan.",
        f"",
        f"We are committed to your child's success, but we need your immediate cooperation.",
        f"",
        f"Best regards,",
        f"School Administration",
        f"",
        f"Note: This is an automated notification. For queries, please contact the school office.",
    ])

    message = "\n".join(message_parts) 
           

attendance_low: True
attendance_low: True
attendance_low: True
attendance_low: True
attendance_low: True
attendance_low: True


In [5]:
print(message,end="\n\n")

Dear Parent/Guardian,

This is an important notice regarding your child, Sneha.

⚠️ WARNING: REQUIRES IMMEDIATE ATTENTION ⚠️

Issue Category: ATTENDANCE

📊 ATTENDANCE CONCERN:
Current Attendance: 68.74% (Required: 80% minimum)
Status: BELOW REQUIRED THRESHOLD

Complete Performance Report:
  • Tamil: 66/100 - ✓ PASS
  • English: 50/100 - ✓ PASS
  • Maths: 62/100 - ✓ PASS
  • Science: 95/100 - ✓ PASS
  • Social: 51/100 - ✓ PASS

Total Marks: 324

Average Marks: 64.80%


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

⚠️ IMPORTANT WARNING ⚠️

If these issues are not addressed and improved in the upcoming evaluation period:

1. The student may face academic probation
2. Promotion to the next grade may be at risk
3. The school will NOT take responsibility for any future academic consequences
4. Additional remedial classes may be mandated at extra cost
5. The student may be required to repeat the current academic year

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

IMMEDIATE ACTIONS REQUIRED:

